In [1]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [5]:
start_date = '2022-10-31'
end_date = '2022-12-11'

In [10]:
query = f"""
with table1 as(
    select 
        yyyymmdd, entity_id, order_unique_id, owner_id,
        cast(offer_type as varchar) as coin_type,
        sum(case when transaction_type = 'credit' then cast(amount as double) end) as coin_credited,
        sum(case when transaction_type = 'debit' and sub_type != 'coinExpired' then cast(amount as double) end) as coin_uitilized,
        sum(case when transaction_type = 'debit' and sub_type = 'coinExpired' then cast(amount as double) end) as coin_expired
    from(
        select
            yyyymmdd, coin_wallet_amount, transaction_type, sub_type, entity_id, owner_id, order_unique_id, coin_wallet_changes,
            -- sub_type,coin_id, order_unique_id, entity_id,
            transform(cast(coin_wallet_changes as array<json>), x-> cast(json_extract(x, '$.offerType') as varchar)) as coin_type,
            row_number() over(partition by id order by updated_epoch desc) as row
        from raw.kafka_payments_transaction_immutable
        where yyyymmdd >= date_format(cast('{start_date}' as date), '%Y%m%d') --]] date_format(current_date - interval '1' day, '%Y%m%d') 
            and yyyymmdd <= date_format(cast('{end_date}' as date), '%Y%m%d') --]] date_format(current_date - interval '1' day, '%Y%m%d')
            and owner_type = 'customer'
            and cardinality(coin_id) != 0
        )
    cross join unnest(coin_type, coin_wallet_amount) as tbl1(offer_type, amount)
    where row = 1
    group by 1,2,3,4,5
),

table2 as(
    select  
        order_id as _id, unique_id as uniqueid, customer_id as customer, service_obj_service_name as serviceobj_service, city_name as serviceobj_city,

        case 
        when (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then 'non_sub' 
        when offer_details_offer_type = 'subscription' then 'subs' 
        else 'test' end customer_types
    from orders.order_logs_snapshot
    where yyyymmdd >= date_format(cast('{start_date}' as date) - interval '14' day, '%Y%m%d') --]] date_format(current_date - interval '60' day, '%Y%m%d') 
        and yyyymmdd <= date_format(cast('{end_date}' as date), '%Y%m%d') --]] date_format(current_date - interval '1' day, '%Y%m%d')
        -- and customer_id in (select distinct owner_id from table1)
        and service_obj_service_name = 'Link'
),

temp1 as(
    select 
        yyyymmdd, coin_type, serviceobj_city, 
        case when (serviceobj_service is null or serviceobj_service = '') then 'Link' else serviceobj_service end as serviceobj_service, 
        entity_id, order_unique_id, owner_id,
        coalesce(customer_types, 'non_sub') customer_types,
        sum(coin_credited) as coin_credited, 
        sum(coin_uitilized) as coin_uitilized, 
        sum(coin_expired) as coin_expired
    from table1
    left join table2 on entity_id = _id 
    where (entity_id is not null and entity_id != '')
    group by 1,2,3,4,5,6,7,8
),

temp2 as(
    select 
        yyyymmdd, coin_type, serviceobj_city, 
        case when (serviceobj_service is null or serviceobj_service = '') then 'Link' else serviceobj_service end as serviceobj_service, 
        entity_id, order_unique_id, owner_id,
        coalesce(customer_types, 'non_sub') customer_types,
        sum(coin_credited) as coin_credited, 
        sum(coin_uitilized) as coin_uitilized, 
        sum(coin_expired) as coin_expired
    from table1
    left join table2 on order_unique_id = uniqueid
    where ((entity_id is null or entity_id = '') and (order_unique_id is not null and order_unique_id != ''))
    group by 1,2,3,4,5,6,7,8
),

temp1_temp2 as(
    select * from temp1
    union
    select * from temp2
),

temp3 as(
    select 
        yyyymmdd, coin_type, 'NA' as serviceobj_city, 
        'Link' as serviceobj_service,
        entity_id, order_unique_id, owner_id, 
        'non_subs' customer_types,
        sum(coin_credited) as coin_credited, sum(coin_uitilized) as coin_uitilized, sum(coin_expired) as coin_expired
    from table1
    where (entity_id is null or entity_id = '') and (order_unique_id is null or order_unique_id = '')
    group by 1,2,3,4,5,6,7
),

temp123 as(
    select * from temp3 
    union 
    (select * from temp1_temp2)
),

user_city as(
    select 
        customerid, registeredcity 
    from datasets.customer_demographics
),

final_table as(
select 
    yyyymmdd, coin_type, 
    case when (serviceobj_city is null or serviceobj_city in ('','NA')) then registeredcity else lower(serviceobj_city) end as city,
    serviceobj_service as service, entity_id as order_id, order_unique_id, owner_id, customer_types, coin_credited, coin_uitilized, coin_expired
from temp123 
left join user_city on owner_id = customerid
),

coins_tally as
(
select
    substr(cast(date_trunc('week',date_parse(yyyymmdd,'%Y%m%d')) as varchar),1,10) StartDate,
    owner_id, 
    -- service, 
    city,
    -- coin_type, 
    sum(coin_credited) as coin_credited, 
    sum(case when customer_types = 'subs' then coin_credited end) as subs_coin_credited,
    sum(coin_expired) as coin_expired,
    sum(case when customer_types = 'subs' then coin_expired end) as subs_coin_expired,
    sum(coin_uitilized) as coin_uitilized, 
    sum(case when customer_types = 'subs' then coin_uitilized end) as subs_coin_uitilized

from final_table

where 1=1
and city in ('hyderabad','delhi','bangalore', 'chennai', 'mumbai','jaipur','kolkata','chandigarh','indore','lucknow')
/*
and coin_type = 'scratchCardOffer'
*/
group by 1,2,3
),

segment as
(
    select
        date_format(date_trunc('week', cast(run_date as date) + interval '1' day), '%Y-%m-%d') start_date,
        substr(cast(date_trunc('week',cast(run_date as date) + interval '1' day) as varchar),1,10) as day,
        (extract(week from cast(run_date as date)) + 1) as week,
        customer_id as customerid,
        case 
        when link_lifetime_stage = 'HANDHOLDING' then '1.HANDHOLDING'
        when link_lifetime_stage = 'HOOK' then '2.HOOK'
        when link_lifetime_stage = 'DETOX' then '3.DETOX'
        when link_lifetime_stage = 'SUSTENANCE' then '4.SUSTENANCE'
        when link_lifetime_stage = 'COMMITTED' then '5.COMMITTED'
        when link_lifetime_stage = 'CHURN_OTB' then '6.CHURN_OTB'
        when link_lifetime_stage = 'SOFT_CHURN' then '7.SOFT_CHURN'
        when link_lifetime_stage = 'DORMANT' then '8.DORMANT'
        when link_lifetime_stage in ('INACTIVE','UNKNOWN') then '9.INACTIVE' 
        end link_lifetime_stage,
        taxi_lifetime_last_ride_city as geo_city

from datasets.iallocator_customer_segments
where 
    cast(run_date as date) >= cast('{start_date}' as date) - interval '1' day 
    and  cast(run_date as date) <= cast('{end_date}' as date)
    and day_of_week(cast(run_date as date)) = 7
    and customer_id in (select distinct owner_id from coins_tally)
    -- and taxi_lifetime_last_ride_city is not null -- NOTE : Data Sets has auto_lifetime_last_ride_city as null also 
),

coins_segment as
(
select 
a.*,
coalesce(b.link_lifetime_stage,'0.NEW') link_lifetime_stage,
b.geo_city
from coins_tally a 
left join segment b on a.owner_id = b.customerid and a.StartDate = b.day
)

select
    StartDate,
    city,
    link_lifetime_stage,
    sum(coin_credited) coin_credited,
    sum(coin_expired) coin_expired,
    (sum(coin_credited) - sum(coin_expired)) coin_burn,
    sum(subs_coin_credited) subs_coin_credited,
    sum(subs_coin_expired) subs_coin_expired,
    (sum(subs_coin_credited) - sum(subs_coin_expired)) subs_coin_burn,
    sum(coin_uitilized) coin_uitilized,
    sum(subs_coin_uitilized) subs_coin_uitilized
from coins_segment
group by 1,2,3
order by 1,2
"""

In [11]:
df = pd.read_sql(query, conn)
df

,StartDate,city,link_lifetime_stage,coin_credited,coin_expired,coin_burn,subs_coin_credited,subs_coin_expired,subs_coin_burn,coin_uitilized,subs_coin_uitilized
0,2022-10-31,bangalore,4.SUSTENANCE,3705.0,2617.00,1088.00,NaN,NaN,NaN,2508.00,10.0
1,2022-10-31,bangalore,7.SOFT_CHURN,1597.0,2408.00,-811.00,NaN,NaN,NaN,1225.00,50.0
2,2022-10-31,bangalore,6.CHURN_OTB,2912.0,4086.00,-1174.00,64.0,NaN,NaN,2181.00,160.0
3,2022-10-31,bangalore,0.NEW,172076.0,8398.00,163678.00,NaN,NaN,NaN,25258.00,NaN
4,2022-10-31,bangalore,3.DETOX,3707.0,1461.00,2246.00,100.0,NaN,NaN,2512.00,460.0
...,...,...,...,...,...,...,...,...,...,...,...
595,2022-12-05,mumbai,0.NEW,101566.0,80.00,101486.00,NaN,NaN,NaN,26937.00,12.0
596,2022-12-05,mumbai,4.SUSTENANCE,8595.0,6292.00,2303.00,NaN,168.0,NaN,7209.00,586.0
597,2022-12-05,mumbai,9.INACTIVE,105127.0,68519.00,36608.00,NaN,4.0,NaN,28510.00,87.0
598,2022-12-05,mumbai,6.CHURN_OTB,4327.0,2788.00,1539.00,127.0,124.0,3.0,2614.00,326.0


In [9]:
df.to_csv("all_coins.csv", index = False)

In [12]:
df.to_csv("all_coins_city.csv", index = False)

In [44]:
df = pd.read_sql(query, conn)
df

,StartDate,service,link_lifetime_stage,coin_credited,subs_coin_credited,coin_expired,subs_coin_expired,coin_burn,subs_coin_burn
0,2022-11-14,Auto,SOFT_CHURN,None,None,42.00,NaN,None,None
1,2022-11-14,Auto,COMMITTED,None,None,150.00,NaN,None,None
2,2022-11-28,Auto,HOOK,None,None,72757.81,1134.0,None,None
3,2022-11-28,Auto,DETOX,None,None,23805.00,585.0,None,None
4,2022-11-28,Auto,CHURN_OTB,None,None,17092.00,330.0,None,None
...,...,...,...,...,...,...,...,...,...
61,2022-11-28,Link,HOOK,None,None,417903.10,7772.0,None,None
62,2022-11-28,Link,NEW,None,None,18525.79,12.0,None,None
63,2022-11-14,Link,DORMANT,None,None,NaN,NaN,None,None
64,2022-11-28,Link,CHURN_OTB,None,None,52392.00,1223.0,None,None


In [45]:
df.to_csv("test.csv", index = False)

In [37]:
df_all_coins = df
df_all_coins

,StartDate,service,link_lifetime_stage,coin_credited,coin_expired,coin_burn
0,2022-11-14,Auto,SUSTENANCE,13267.0,7322.00,5945.00
1,2022-11-21,Auto,INACTIVE,124152.0,224602.51,-100450.51
2,2022-11-14,Auto,UNKNOWN,178202.0,259671.00,-81469.00
3,2022-11-28,Auto,COMMITTED,30456.0,39477.76,-9021.76
4,2022-11-28,Auto,SOFT_CHURN,8700.0,17842.21,-9142.21
...,...,...,...,...,...,...
61,2022-11-21,Link,SUSTENANCE,166754.0,124597.94,42156.06
62,2022-11-28,Link,CHURN_OTB,131843.0,67477.00,64366.00
63,2022-11-28,Link,SOFT_CHURN,69628.0,30110.50,39517.50
64,2022-11-21,Link,HOOK,610734.0,657869.37,-47135.37


In [35]:
df.to_csv("all_coins.csv", index = False)